In [64]:
# GUI for jupyter: https://docs.bamboolib.8080labs.com/

import pandas
import numpy as np
import requests
from config import AV_API_KEY

STOCK_SYMBOL = 'INTC'
AV_URL = "https://www.alphavantage.co/query" # Alpha Vantage API call: 


def get_from_av(sym, query_type):
    return requests.get(AV_URL, params={
        "function": query_type,
        "symbol": sym,
        "apikey": AV_API_KEY
    }).json()

def get_stock_data(sym):
    '''retrieve data from Alpha Vantage: OVERVIEW, BALANCE_SHEET, TIME_SERIES_MONTHLY_ADJUSTED'''
    data = {}
    query_types = []
    data['overview'] = get_from_av(sym, 'OVERVIEW')
    data['income'] = get_from_av(sym, 'INCOME_STATEMENT')['annualReports']
    data['balance'] = get_from_av(sym, 'BALANCE_SHEET')['annualReports']
    data['monthly_quotes'] = get_from_av(sym, 'TIME_SERIES_MONTHLY_ADJUSTED')['Monthly Adjusted Time Series']
    return data

data = None
data = get_stock_data(STOCK_SYMBOL)

import json
print(json.dumps(data, indent=2))


{
  "overview": {
    "Symbol": "INTC",
    "AssetType": "Common Stock",
    "Name": "Intel Corporation",
    "Description": "Intel Corporation designs, manufactures, and sells essential technologies for the cloud, smart, and connected devices worldwide. The company operates through DCG, IOTG, Mobileye, NSG, PSG, CCG, and All Other segments. It offers platform products, such as central processing units and chipsets, and system-on-chip and multichip packages; and non-platform or adjacent products comprising accelerators, boards and systems, connectivity products, and memory and storage products. The company also provides Internet of things products, including high-performance compute solutions for targeted verticals and embedded applications; and computer vision and machine learning-based sensing, data analysis, localization, mapping, and driving policy technology. It serves original equipment manufacturers, original design manufacturers, and cloud service providers. The company has col

In [65]:
def format_monthly_quote(quote):
    # Removes the prefix number and converts values to int
    # "1. open": "59.9100"
    formatted = {}
    for key in quote:
        cleaned_key = key.split(". ")[1]
        formatted[cleaned_key] = float(quote[key])
    return formatted

def restructure_data(data):
    '''
    Flattens the JSON structure
    1. Reduces monthly quotes to last month of each year,
    2. Combines balance sheets, income statements, and monthly quotes into a single dictionary 
       {'2019-12-31': {balance_sheet, income_statement, monthly_quote}, ...}
    '''
    final_data = {}

    # Gathers the monthly stock prices (Groups by year YYYY)
    monthly_close_prices = {}
    for date in data['monthly_quotes']:
        current_year = date.split("-")[0]
        cleaned_quote = format_monthly_quote(data['monthly_quotes'][date])
        if current_year not in monthly_close_prices:
            monthly_close_prices[current_year] = []
        monthly_close_prices[current_year].append(cleaned_quote['close'])
    
    # Calculate the average stock price for each year
    for year in monthly_close_prices.keys():
        year_avg_price = round(sum(monthly_close_prices[year]) / len(monthly_close_prices[year]), 2)
        # print("Average price for ", year, ": ", year_avg_price)
        final_data[year] = {"average_price": year_avg_price}

    # Add the balance sheet and income statement for each year to final
    for balance, income in list(zip(data['balance'], data['income'])):
        if balance['fiscalDateEnding'] == income['fiscalDateEnding']:
            current_year = balance['fiscalDateEnding'].split("-")[0]
            print("current_year: ", current_year)
            financials = {**balance, **income}
            final_data[current_year].update(financials)


    return final_data

overview_data = data['overview']
overview_data['fair_values'] = {} # NOTE: HISTORICAL FAIR VALUES WILL BE STORED HERE
historical_data = restructure_data(data)

# Load data into pandas dataframe 
df = pandas.read_json(json.dumps(historical_data), orient="index", )

# Drop Years that are missing data 
# Balance sheets and Income Statements only give 5 yrs data compared to Monthly Quote data
df = df.dropna(axis=0)
df

current_year:  2019
current_year:  2018
current_year:  2017
current_year:  2016
current_year:  2015


,accountsPayable,accumulatedAmortization,accumulatedDepreciation,additionalPaidInCapital,average_price,capitalLeaseObligations,capitalSurplus,cash,cashAndShortTermInvestments,commonStock,...,totalLongTermDebt,totalNonCurrentAssets,totalNonCurrentLiabilities,totalOperatingExpense,totalOtherIncomeExpense,totalPermanentEquity,totalRevenue,totalShareholderEquity,treasuryStock,warrants
2019,7.981000e+09,None,None,None,51.72,None,None,3.276000e+09,1.220500e+10,2.526100e+10,...,2.530800e+10,5.538600e+10,3.518700e+10,2.010500e+10,1.636000e+09,None,7.196500e+10,7.750400e+10,None,155000000.0
2018,7.446000e+09,None,None,None,49.41,None,None,6.407000e+09,1.503800e+10,2.536500e+10,...,2.509800e+10,4.897600e+10,3.430600e+10,2.042100e+10,1.030000e+08,None,7.084800e+10,7.456300e+10,None,419000000.0
2017,6.454000e+09,None,None,0,38.35,None,None,7.145000e+09,1.771400e+10,2.607400e+10,...,2.503700e+10,4.110900e+10,3.594300e+10,2.113300e+10,2.237000e+09,0,6.276100e+10,6.901900e+10,0,866000000.0
2016,6.750000e+09,None,None,0,33.50,None,None,5.560000e+09,1.709900e+10,2.537300e+10,...,2.064900e+10,7.781900e+10,2.591700e+10,2.331700e+10,-1.313000e+09,0,5.938700e+10,6.622600e+10,0,882000000.0
2015,6.161000e+09,None,None,0,32.14,None,None,1.530800e+10,2.531300e+10,2.341100e+10,...,2.003600e+10,6.270900e+10,2.541600e+10,2.067700e+10,-1.440000e+08,0,5.535500e+10,6.108500e+10,0,897000000.0


In [66]:
# Calculate EPS for each year
# TODO: Need to account for 'None' in preferredStockTotalEquity
# df["eps"] = round( (df['netIncome'] - df['preferredStockTotalEquity']) / float(df['commonStockSharesOutstanding']), 2)
df["eps"] = round(df['netIncome'] / df['commonStockSharesOutstanding'], 2)
df["eps"]

2019    4.91
2018    4.66
2017    2.05
2016    2.18
2015    2.42
Name: eps, dtype: float64

In [67]:
# Calculate each yr's PE Ratio and the historical average
df['peRatio'] = round(df['average_price'] / df['eps'], 2)
print(df['peRatio'])
pe_ratios = list(pandas.array(df['peRatio']))
overview_data['historical_peratio'] = round(sum(pe_ratios) / len(pe_ratios), 2)
print("average historical PE Ratio: ", overview_data['historical_peratio'])

# Calculate average historical price from historical_peratio
eps_array = list(pandas.array(df['eps']))
print(eps_array)
# overview_data['fair_price_from_peratio'] = overview_data['historical_peratio'] * round(sum(eps_array) / len(eps_array), 2)
pe_ratio_fair_value = overview_data['historical_peratio'] * round(sum(eps_array) / len(eps_array), 2)
overview_data['fair_values']['PERatio'] = pe_ratio_fair_value

print("Stock Price from historical PE Ratio: ", overview_data['fair_values']['PERatio'])

2019    10.53
2018    10.60
2017    18.71
2016    15.37
2015    13.28
Name: peRatio, dtype: float64
average historical PE Ratio:  13.7
[4.91, 4.66, 2.05, 2.18, 2.42]
Stock Price from historical PE Ratio:  44.388


In [68]:
# Calculate Historical Fair Value (11 years) from:
# 5-yr average yield
# 12-yr median yield
# Earnings
# Owner earnings
# Operating cash flow
# Free cash flow
# EBITDA
# EBIT
## Then average these all together

# Calculate share price from Operating Cash Flow
# operating_cash_flow = net_income + depreciation - accounts_receivable - accounts_payable
df['operating_cash_flow'] = df['netIncome'] + df['accumulatedDepreciation'] - (df['accountsPayable'] + df['netReceivables'])
print(df['accumulatedDepreciation'])
print(df['operating_cash_flow']) # TODO WHY IS 2017 -10061999999 ????




# # calculate equity value from EV/EBITDA
# # https://www.wallstreetoasis.com/forums/evebidta-to-target-price#:~:text=With%20the%20EV%2FEBITDA%20multiple,the%20equity%20value%20per%20share.
# enterprise_value = float(data['overview']['EVToEBITDA']) * float(data['overview']['EBITDA'])
# total_debt = float(data['annual_reports'][0]['shortTermDebt']) + float(data['annual_reports'][0]['longTermDebt'])
# equity_value = float(enterprise_value) - float(total_debt)
# share_price = round(float(equity_value) / float(data['overview']['SharesOutstanding']), 2)
# print("SHARE PRICE from EBITDA: ", share_price) # NOTE: SEEMS TO BE CURRENT YEAR. 

# #TODO: Calculate share price from EBITDA for past 11-yrs

TypeError: unsupported operand type(s) for +: 'float' and 'str'